In [1]:
import torch
from tqdm import tqdm
import json, os, sys
from rouge import Rouge
from transformers import AutoTokenizer

os.chdir('/workspace')
sys.path.append('src')
device = 'cuda'
ckpt_list = [_.split('-')[-1] for _ in os.listdir('/workspace/ckpt/kobart_ckpt') if _.startswith('checkpoint')]
ckpt_list.sort()

test_dataset = json.load(open('data/article/valid_dataset.json', 'r', encoding='utf-8'))
tokenizer = AutoTokenizer.from_pretrained('models/kobart')
rouge_scorer = Rouge()
target_list = [target['abs'] for target in test_dataset]

In [10]:

ckpt = 25000

result_list = []
model_path = f'/workspace/ckpt/kobart_ckpt/checkpoint-{ckpt}'
model = torch.load(os.path.join(model_path, 'multitask_ext_abs_summary_model.pt')).model.to(device)
print(f"Current doing... {model_path.split('/')[-1]}")

def infer(data):
    doc = ' '.join(data['sentences'])
    input_ids = tokenizer(doc, return_tensors="pt").input_ids.to(device)

    output = model.generate(input_ids, num_beams=5, eos_token_id=1, max_length=200, 
                            repetition_penalty=1.2, no_repeat_ngram_size=1, early_stopping=True,
                            length_penalty=0, forced_eos_token_id=1
                             )
    print(len(output[0]))
    result = tokenizer.decode(output[0], skip_special_toknes=True).replace('</s>', '')  
    return result

Current doing... checkpoint-25000


In [11]:
idx = 8
result = infer(test_dataset[idx])
result

200


' 차량용 잠금장치와 관련 부품을 생산하는 평화정공은 현대자동차와 기아자동차의 신차 효과가 기대되는 가운데 중국 정부의 경기부양책에 따른 소비 증가 효과로 인해 올해 영업이익이 320억원으로 지난해보다 60% 늘어날 것으로 전망되어 투자수요를 끌어모으고 있는 가운데, 이베스트투자증권 연구원은 "1분기에 지난해 같은 기간보다 9.1% 늘어난 65억원의 영업이익을 냈다고 추정된다"고 밝혔다.  \n\n현대차, 기아차 등 신규 스포츠유틸리티차량(SUV)의 인기가 높아지는 점이 긍정적 영향을 주고 있고 현대차 노조도 지난달 미국 판매는 전년 동기 대비 10.2% 늘었으며 현대·기아차의 생산라인 증산에 합의해 부품 주문이 더 증가할 것이란 관측이 나오기 때문"이라는 분석을 내 놓았으며, 이에 따라 주가가 0.37배밖에 되지 않는 주가순자산비율(PBR:주가/주당 순자산을 제외해도) 1.3% 미만인 낮은 수치를 보이고 있어 수익성이 개선될 것이라는 전망이 핵심 상승요인으로 작용하고 있다.'

In [13]:
test_dataset[8]

{'ext': ['8일 코스닥시장에서 평화정공은 190원(1.97%) 오른 9850원에 마감했다.',
  '지난해를 바닥으로 올해부터 실적이 본격 개선될 것이란 전망이 핵심 상승요인으로 작용했다.',
  '현대차의 팰리세이드, 기아차의 텔루라이드 등 신규 스포츠유틸리티차량(SUV)의 인기가 높아지고 있는 점이 긍정적 영향을 주고 있다.'],
 'abs': '현대자동차와 기아자동차가 미국에서 좋은 성적을 냄에 따라 평화정공의 실적이 개선될 것이라는 전망이 호재로 작용해 8일 평화정공은 코스닥시장에서 올 들어 47.67% 상승한 9850원으로 마감했다.',
 'sentences': ['] 차량용 잠금장치와 관련 부품을 생산하는 평화정공이 실적개선 기대에 힘입어 투자수요를 끌어모으고 있다.',
  '납품기업인 현대자동차와 기아자동차의 신차 효과가 기대되는 가운데 중국 정부의 경기부양책에 따른 소비 증가 효과가 본격화될 것이란 전망에 힘이 실리고 있다.',
  '8일 코스닥시장에서 평화정공은 190원(1.97%) 오른 9850원에 마감했다.',
  '올해 상승률은 47.67%에 달한다.',
  '지난해를 바닥으로 올해부터 실적이 본격 개선될 것이란 전망이 핵심 상승요인으로 작용했다.',
  '이베스트투자증권에 따르면 올해 평화정공의 영업이익은 320억원으로 지난해보다 60% 늘어날 것으로 전망된다.',
  '현대차의 팰리세이드, 기아차의 텔루라이드 등 신규 스포츠유틸리티차량(SUV)의 인기가 높아지고 있는 점이 긍정적 영향을 주고 있다.',
  '기아차의 지난달 미국 판매는 전년 동기 대비 10.2% 늘었다.',
  '현대차 노조도 지난 1일 팰리세이드 증산에 합의해 부품 주문이 더 늘 것이란 관측이 나온다.',
  '유지웅 이베스트투자증권 연구원은 "1분기부터 평화정공 미국법인 가동률이 개선되기 시작했다"며 "1분기에 지난해 같은 기간보다 9.1% 늘어난 65억원의 영업이익을 낸 것으로 추정된다"고 말했다.',
  '금융정보업체 에프앤가이드에 따르면 평화정공의 주가